현진: 호기심
원준: 플래그/폴리시
나: 예측 기반 오차/오류





에피소드 기반 보상은 나중에

In [10]:
#에러기반
import re

#  오류 유형별 penalty 상수 선언
PENALTY_DICT = {
    'hostname_resolution_error':   -0.2,  # 호스트 이름 해석 실패
    'device_access_error':         -0.3,  # 장치 접근 권한/불가
    'network_unreachable':         -0.25, # 대상 네트워크 접근 불가
    'invalid_target':              -0.15, # 입력 타겟값 부적절
    'nmap_internal_error':         -0.4   # 내부 오류(비정상 종료 등)
}
MIN_REWARD = -1.0  # 최저 보상값 (절대치)
MAX_REWARD = 1.0   # 최고 보상값

def error(output_log):
    """
    nmap 실행 중 발생한 오류 메시지를 분석하고,
    자동으로 수정 가능한 경우 대응 조치를 제안하거나 보정한다.

    Parameters:
    - output_log (str): nmap 실행 결과 로그 문자열

    Returns:
    - dict: {
        'error_detected': bool,      # 오류 존재 여부
        'error_type': str,           # 오류 유형 식별(키)
        'suggested_fix': str | None, # 제안된 수정 또는 재시도 방안
        'auto_fixable': bool         # 자동 수정 가능 여부
      }
    """
    result = {
        'error_detected': False,
        'error_type': None,
        'suggested_fix': None,
        'auto_fixable': False
    }
    log = output_log.lower()

    # 호스트 이름 해석 실패
    if re.search(r"(unable to resolve hostname|dns resolution failed|error resolving name)", log):
        result.update({
            'error_detected': True,
            'error_type': 'hostname_resolution_error',
            'suggested_fix': '대상 호스트 이름이 올바른지 확인하거나 IP 주소로 직접 입력하세요.',
            'auto_fixable': True
        })
    #  장치 접근 불가
    elif "failed to open device" in log:
        result.update({
            'error_detected': True,
            'error_type': 'device_access_error',
            'suggested_fix': '루트 권한으로 다시 실행하거나 네트워크 인터페이스 이름을 명시하세요. (예: nmap -e eth0 ...)',
            'auto_fixable': False
        })
    #  대상 접근 불가
    elif "no route to host" in log:
        result.update({
            'error_detected': True,
            'error_type': 'network_unreachable',
            'suggested_fix': '대상 네트워크 연결 상태를 점검하고, 방화벽 또는 VPN 설정을 확인하세요.',
            'auto_fixable': False
        })
    #  유효하지 않은 대상
    elif "not a valid target" in log:
        result.update({
            'error_detected': True,
            'error_type': 'invalid_target',
            'suggested_fix': '입력한 대상 형식(IP, CIDR 등)을 다시 확인하세요.',
            'auto_fixable': True
        })
    #  nmap 내부 오류 (일반적 종료)
    elif re.search(r"(nmap error|quitting)", log):
        result.update({
            'error_detected': True,
            'error_type': 'nmap_internal_error',
            'suggested_fix': 'nmap 명령 구문과 옵션을 다시 확인하거나 -d(디버그 모드)로 재실행하세요.',
            'auto_fixable': False
        })
    #  오류 없음
    else:
        result.update({
            'error_detected': False,
            'error_type': None,
            'suggested_fix': None,
            'auto_fixable': False
        })
    return result

def calc_penalty(log):
    """
    error() 함수로 식별된 오류 유형에 따라 해당 penalty만큼
    보상을 감소시키는 단일 책임 함수. 적용 penalty는 최저/최고 보상값 범위 내로 제한.
    """
    err = error(log)  # 오류 분석 실행
    # 오류 유형이 발견되면 PENALTY_DICT 값을 적용, 아니면 0(감점 없음)
    penalty = PENALTY_DICT.get(err['error_type'], 0.0)
    # 총 보상(예: 1.0에서 penalty만큼 빼기, 실제 시스템에서는 누적 보상/스텝별 반영)
    reward = max(MIN_REWARD, min(MAX_REWARD, 1.0 + penalty))
    # 상세 결과표와 계산된 보상 함께 반환
    return {'penalty': penalty, 'reward': reward, **err}

# 예시 사용
log1 = "Nmap scan report: Unable to resolve hostname example.local"
print(calc_penalty(log1))


{'penalty': -0.2, 'reward': 0.8, 'error_detected': True, 'error_type': 'hostname_resolution_error', 'suggested_fix': '대상 호스트 이름이 올바른지 확인하거나 IP 주소로 직접 입력하세요.', 'auto_fixable': True}


In [4]:
# phylo_reward_demo.py
from typing import Dict, Optional, Tuple
import math
import time

Node = str

# -----------------------------
# 간단 트리: LCA/거리/보상
# -----------------------------
class SimpleTree:
    def __init__(self, parent: Dict[Node, Optional[Node]], depth: Dict[Node, int]):
        self.parent = parent
        self.depth = depth

    def _align_depth(self, u: Node, v: Node) -> Tuple[Node, Node]:
        du, dv = self.depth[u], self.depth[v]
        while du > dv:
            u = self.parent[u]; du -= 1
        while dv > du:
            v = self.parent[v]; dv -= 1
        return u, v

    def lca(self, u: Node, v: Node) -> Node:
        u, v = self._align_depth(u, v)
        path = []
        while u != v:
            path.append((u, v))
            u = self.parent[u]
            v = self.parent[v]
        return u

    def dist(self, u: Node, v: Node) -> int:
        a = self.lca(u, v)
        return (self.depth[u] - self.depth[a]) + (self.depth[v] - self.depth[a])

def reward_inverse_distance(d: int) -> float:
    return 1.0 / (1 + d)

def reward_with_case_weights(d: int, weights: Dict[int, float]) -> float:
    return weights.get(d, 1.0 / (1 + d))

# -----------------------------
# 로거(깔끔한 콘솔 출력)
# -----------------------------
def log(section: str, msg: str = ""):
    ts = time.strftime("%H:%M:%S")
    print(f"[{ts}] {section:<10} | {msg}")

def assert_eq(name: str, got, expect):
    ok = got == expect
    status = "PASS" if ok else "FAIL"
    log("TEST", f"{name:<28} -> {status} (got={got!r}, expect={expect!r})")
    return ok

def assert_close(name: str, got: float, expect: float, tol: float = 1e-9):
    ok = abs(got - expect) <= tol * max(1.0, abs(expect))
    status = "PASS" if ok else "FAIL"
    log("TEST", f"{name:<28} -> {status} (got={got:.12f}, expect={expect:.12f})")
    return ok

# -----------------------------
# 데모 트리(루트→프로그램→명령어→리프)
# 깊이: R(0) -> P(1) -> C(2) -> L(3)
# -----------------------------
def build_demo_tree() -> SimpleTree:
    parent = {
        "R": None,
        "git": "R",
        "git:status": "git",
        "git:commit": "git",
        # 리프
        "L_status_v1": "git:status",
        "L_status_v2": "git:status",
        "L_commit_simple": "git:commit",
    }
    depth = {
        "R": 0,
        "git": 1,
        "git:status": 2,
        "git:commit": 2,
        "L_status_v1": 3,
        "L_status_v2": 3,
        "L_commit_simple": 3,
    }
    return SimpleTree(parent, depth)

# -----------------------------
# 시나리오 실행 & 로깅
# -----------------------------
def run_demo():
    log("START", "Phylo reward demo (no pytest)")
    tree = build_demo_tree()

    # 거리 테스트
    log("CASE", "거리 d 테스트")
    d00 = tree.dist("L_status_v1", "L_status_v1")  # 0
    d02 = tree.dist("L_status_v1", "L_status_v2")  # 2 (형제)
    d04 = tree.dist("L_status_v1", "L_commit_simple")  # 4 (사촌, LCA=git)

    assert_eq("dist(same leaf)", d00, 0)
    assert_eq("dist(sibling leaves)", d02, 2)
    assert_eq("dist(cousin leaves)", d04, 4)

    # LCA 테스트
    log("CASE", "LCA 테스트")
    a1 = tree.lca("L_status_v1", "L_status_v2")
    a2 = tree.lca("L_status_v1", "L_commit_simple")
    assert_eq("lca(status siblings)", a1, "git:status")
    assert_eq("lca(status vs commit)", a2, "git")

    # 보상: 역수형
    log("CASE", "보상 r=1/(1+d) 테스트")
    r0 = reward_inverse_distance(d00)  # 1.0
    r2 = reward_inverse_distance(d02)  # 1/3
    r4 = reward_inverse_distance(d04)  # 1/5
    assert_close("reward(d=0)", r0, 1.0)
    assert_close("reward(d=2)", r2, 1.0/3.0)
    assert_close("reward(d=4)", r4, 1.0/5.0)

    # 보상: 케이스별 상수(weight 테이블)
    log("CASE", "케이스별 상수 보상(테이블)")
    weights = {0: 1.0, 2: 0.40, 3: 0.22}  # 예시: d=2는 0.40으로 덮어쓰기
    rw2 = reward_with_case_weights(d02, weights)
    rw4 = reward_with_case_weights(d04, weights)  # 테이블에 없으니 역수형 폴백(0.2)
    assert_close("case-weight(d=2)", rw2, 0.40)
    assert_close("case-weight(d=4)", rw4, 1.0/5.0)

    # 요약 출력
    log("SUMMARY", f"d=0 -> r={r0:.3f} | d=2 -> r={r2:.3f} (or {rw2:.3f} by table) | d=4 -> r={r4:.3f}")

    log("DONE", "All checks complete.")

if __name__ == "__main__":
    run_demo()


[18:16:11] START      | Phylo reward demo (no pytest)
[18:16:11] CASE       | 거리 d 테스트
[18:16:11] TEST       | dist(same leaf)              -> PASS (got=0, expect=0)
[18:16:11] TEST       | dist(sibling leaves)         -> PASS (got=2, expect=2)
[18:16:11] TEST       | dist(cousin leaves)          -> PASS (got=4, expect=4)
[18:16:11] CASE       | LCA 테스트
[18:16:11] TEST       | lca(status siblings)         -> PASS (got='git:status', expect='git:status')
[18:16:11] TEST       | lca(status vs commit)        -> PASS (got='git', expect='git')
[18:16:11] CASE       | 보상 r=1/(1+d) 테스트
[18:16:11] TEST       | reward(d=0)                  -> PASS (got=1.000000000000, expect=1.000000000000)
[18:16:11] TEST       | reward(d=2)                  -> PASS (got=0.333333333333, expect=0.333333333333)
[18:16:11] TEST       | reward(d=4)                  -> PASS (got=0.200000000000, expect=0.200000000000)
[18:16:11] CASE       | 케이스별 상수 보상(테이블)
[18:16:11] TEST       | case-weight(d=2)             -> PASS

In [ ]:
#뭐가 어떻게 될지 몰라 일단 함수 기틀만 만들어 보았음.
'''우선, 호기심 보상 함수를 두가지 버전으로 만들어보앗는데,
첫번째로는 은세가 말했던 스텝이 진행될 수록 기틀이 잡히니까 호기심 보상을 자동적으로 감쇠시키는 버전,
두번째로는 후반에 가서도 호기심 보상을 감소시키지 않을 수 있게 최악의 상태일떄 음의 보상으로 패치하는 버전으로 만들었음.
'''


#자동 감쇠형 호기심 함수
def curiosity_reward_decay(step, base_reward=1.0, decay_rate=0.005):
    """
    호기심 보상이 스텝(step)에 따라 자연스럽게 감소하는 함수.
    예: 초반엔 높은 보상, 후반으로 갈수록 점점 감소.

    R_c(t) = base_reward * (1 - decay_rate * step)
    (단, 0보다 작으면 0으로 제한)
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    return reward
# 최악의 경우에 음의 보상을 주는 호기심 함수
def curiosity_reward_with_penalty(is_redundant=False, is_error=False, step=0,
                                  base_reward=1.0, penalty_redundant=0.3, penalty_error=0.5,
                                  decay_rate=0.003):
    """
    스텝 수에 따른 호기심 보상 감소 + 잘못된 행동에 대한 음의 보상 적용.

    Parameters:
    - is_redundant: 이미 여러 번 시도된 동일한 행동 수행시 True ---
    - is_error: 시스템 오류를 유발하는 탐색시 True >>>
    - step: 현재 스텝 수 >>>
    - base_reward: 기본 호기심 보상 >>>
    - penalty_redundant: 반복 행동에 부여할 음의 보상 강도
    - penalty_error: 심각한 오류 행위에 부여할 음의 보상 강도
    - decay_rate: 스텝에 따른 감쇠 비율
    """
    reward = max(base_reward * (1 - decay_rate * step), 0)
    if is_redundant:
        reward -= penalty_redundant
    if is_error:
        reward -= penalty_error
    return max(reward, -1.0)



# 새로운 행동 -> 보상

# 시간 감쇠

# 연속 반복 페널티


In [ ]:
import hashlib
import hmac

def flag_reward(flag_str, known_flags, big_reward=1000.0):
    """
    플래그 정답 여부에 따른 스칼라 보상 계산 함수 (순수 함수)
    
    파라미터:
    - flag_str     : (str) 제출/발견된 플래그 원문
    - known_flags  : (dict) {"FLAG": "<sha256_hex>"}  (정답 해시 딕셔너리)
    - big_reward   : (float) 정답시 지급 보상 크기 (기본 1000.0)
    
    반환값:
    - reward       : (float) 정답 시 big_reward, 아니면 0.0
    """

    # 입력 검증
    if not isinstance(flag_str, str):
        return 0.0

    # 제출 플래그 해시 계산
    submitted_hash = hashlib.sha256(flag_str.strip().encode("utf-8")).hexdigest()

    # 등록된 정답 해시 확인
    try:
        expected_key, expected_hash = next(iter(known_flags.items()))
    except StopIteration:
        return 0.0

    # 안전 비교
    is_correct = hmac.compare_digest(submitted_hash, expected_hash)

    # 보상 산출 (오직 정답만 지급, 외부 상태 미변경)
    reward = float(big_reward) if is_correct else 0.0

    # [PARSE_HOOK]  
    # 필요하면 여기서 플래그 내 추가 메타데이터(REWARD= 등) 파싱 확장
    # 예시:
    #   parsed_reward = parse_reward_from_string(flag_str)
    #   if is_correct and parsed_reward is not None:
    #       reward += float(parsed_reward)

    return reward


In [ ]:
def main():   
    error_reward()
    prophecy_reward()
    flag_reward()
    curiosity()
    return

In [ ]:
# Policy (ε-탐욕 기반)
def policy_update(actions, Q, epsilon=0.1, alpha=0.1, last_action=None, reward=0):
    """
    간단한 ε-탐욕 정책 함수 (기틀 버전)
    - actions: 가능한 행동 리스트
    - Q: 행동 가치 테이블 (dict)
    - epsilon: 무작위 탐색 확률
    - alpha: 학습률
    - last_action: 직전 수행한 행동
    - reward: 직전 보상
    """
    import random

    # Q값 업데이트 (최근 보상 반영)
    if last_action is not None:
        old_value = Q.get(last_action, 0)
        Q[last_action] = old_value + alpha * (reward - old_value)

    # ε-탐욕 정책으로 다음 행동 선택
    if random.random() < epsilon:
        next_action = random.choice(actions)
    else:
        next_action = max(actions, key=lambda a: Q.get(a, 0))

    return next_action